In [ ]:
%matplotlib inline
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import flopy
sys.version

In [ ]:
model_ws = os.path.join('..', 'test041_flowdivert_nwt_dev', 'mfnwt')

#if not os.path.isdir(model_ws):
#    os.mkdir(model_ws)

# load existing model
m = flopy.modflow.Modflow.load('flowdivert.nam', version='mfnwt', verbose=True, model_ws=model_ws)

## MODFLOW 6

In [ ]:
mf6_ws = os.path.join('..', 'test041_flowdivert_nwt_dev')
hdobj = flopy.utils.binaryfile.HeadFile(os.path.join(mf6_ws, 'flowdivert.hds'))
head6 = hdobj.get_data()
bdobj = flopy.utils.binaryfile.CellBudgetFile(os.path.join(mf6_ws, 'flowdivert.cbc'), precision='double')
bdobj.list_records()
#frf6 = bdobj.get_data(text='FLOW RIGHT FACE')[0]
#fff6 = bdobj.get_data(text='FLOW FRONT FACE')[0]

In [ ]:
# Plot head and flow vectors
fig = plt.figure(figsize=(15,15))
mm = flopy.plot.ModelMap(model=m)
mm.plot_grid()
mm.plot_ibound()
qm = mm.plot_array(head6[0])
cb = plt.colorbar(qm, shrink=0.5)

In [ ]:
# Plot saturated thickness and flow vectors
fig = plt.figure(figsize=(15,15))
mm = flopy.plot.ModelMap(model=m)
mm.plot_grid()
mm.plot_ibound()
sat_thk = head6[0] - m.dis.botm.array[0]
sat_thk = np.where(sat_thk < 0.01, 0, sat_thk)
qm = mm.plot_array(sat_thk, masked_values=[0])
cb = plt.colorbar(qm, shrink=0.5)

# MODFLOW-NWT

In [ ]:
mfnwt_ws = os.path.join('..', 'test041_flowdivert_nwt_dev', 'mfnwt')
hdobj = flopy.utils.binaryfile.HeadFile(os.path.join(mfnwt_ws, 'flowdivert.hds'))
headnwt = hdobj.get_data()
bdobj = flopy.utils.binaryfile.CellBudgetFile(os.path.join(mfnwt_ws, 'flowdivert.cbc'), precision='double')
bdobj.list_records()
frfusg = bdobj.get_data(text='FLOW RIGHT FACE')[0]
fffusg = bdobj.get_data(text='FLOW FRONT FACE')[0]

In [ ]:
# Plot head and flow vectors
fig = plt.figure(figsize=(15,15))
mm = flopy.plot.ModelMap(model=m)
mm.plot_grid()
mm.plot_ibound()
qm = mm.plot_array(headnwt[0])
mm.plot_discharge(frfusg, fffusg, head=head6)
cb = plt.colorbar(qm, shrink=0.5)

In [ ]:
# Plot saturated thickness and flow vectors
fig = plt.figure(figsize=(15,15))
mm = flopy.plot.ModelMap(model=m)
mm.plot_grid()
mm.plot_ibound()
sat_thk = headnwt[0] - m.dis.botm.array[0]
#print(sat_thk.min(), sat_thk.max())
sat_thk = np.where(sat_thk < 0.01, 0, sat_thk)
qm = mm.plot_array(sat_thk, masked_values=[0])
mm.plot_discharge(frfusg, fffusg, head=headnwt)
cb = plt.colorbar(qm, shrink=0.5)

# Difference (MODFLOW 6 - MODFLOW-NWT)

In [ ]:
fig = plt.figure(figsize=(15,15))
mm = flopy.plot.ModelMap(model=m)
mm.plot_grid()
mm.plot_ibound()
diff = head6 - headnwt
print(diff.min(), diff.max())
qm = mm.plot_array(diff, masked_values=[0])
#mm.plot_discharge(frf6, fff6, head=head6)
cb = plt.colorbar(qm, shrink=0.5)

In [ ]:
err = np.where(sat_thk == 0, 0., diff)
print(err.min(), err.max())

In [ ]:
fig = plt.figure(figsize=(15,15))
mm = flopy.plot.ModelMap(model=m)
mm.plot_grid()
mm.plot_ibound()
diff = head6 - headnwt
print(diff.min(), diff.max())
qm = mm.plot_array(err)
cb = plt.colorbar(qm, shrink=0.5)